System information (for reproducibility):

In [5]:
versioninfo()

Julia Version 1.9.0
Commit 8e63055292 (2023-05-07 11:25 UTC)
Platform Info:
  OS: Windows (x86_64-w64-mingw32)
  CPU: 16 × AMD Ryzen 7 5800 8-Core Processor              
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-14.0.6 (ORCJIT, znver3)
  Threads: 9 on 16 virtual cores
Environment:
  JULIA_NUM_THREADS = 8


Load packages:

In [6]:
using Pkg

Pkg.activate(pwd())
Pkg.instantiate()
Pkg.status()

  Activating project at `C:\Users\larry\Dropbox\zza\UCLA\academic\year 3\quarter 3\257\biostat-257-2023-spring\hw6`
Precompiling project...
  ✓ KNITRO
  1 dependency successfully precompiled in 78 seconds. 127 already precompiled.

Status `C:\Users\larry\Dropbox\zza\UCLA\academic\year 3\quarter 3\257\biostat-257-2023-spring\hw6\Project.toml`
  [6e4b80f9] BenchmarkTools v1.3.2
  [336ed68f] CSV v0.10.10
  [a93c6f00] DataFrames v1.5.0
  [8bb1440f] DelimitedFiles v1.9.1
  [31c24e10] Distributions v0.25.95
  [b6b21f68] Ipopt v1.4.0
  [67920dd8] KNITRO v0.13.2
  [b8f27783] MathOptInterface v1.17.0
  [ff71e718] MixedModels v4.14.1
  [76087f3c] NLopt v0.6.5
  [08abe8d2] PrettyTables v2.2.4
  [6f49c342] RCall v0.13.15
  [37e2e46d] LinearAlgebra
  [9a3f8284] Random


In this assignment, we continue with the linear mixed effects model (LMM) considered in HW3
$$
    \mathbf{Y}_i = \mathbf{X}_i \boldsymbol{\beta} + \mathbf{Z}_i \boldsymbol{\gamma}_i + \boldsymbol{\epsilon}_i, \quad i=1,\ldots,n,
$$
where   
- $\mathbf{Y}_i \in \mathbb{R}^{n_i}$ is the response vector of $i$-th individual,  
- $\mathbf{X}_i \in \mathbb{R}^{n_i \times p}$ is the fixed effects predictor matrix of $i$-th individual,  
- $\mathbf{Z}_i \in \mathbb{R}^{n_i \times q}$ is the random effects predictor matrix of $i$-th individual,  
- $\boldsymbol{\epsilon}_i \in \mathbb{R}^{n_i}$ are multivariate normal $N(\mathbf{0}_{n_i},\sigma^2 \mathbf{I}_{n_i})$,  
- $\boldsymbol{\beta} \in \mathbb{R}^p$ are fixed effects, and  
- $\boldsymbol{\gamma}_i \in \mathbb{R}^q$ are random effects assumed to be $N(\mathbf{0}_q, \boldsymbol{\Sigma}_{q \times q}$) independent of $\boldsymbol{\epsilon}_i$.

The log-likelihood of the $i$-th datum $(\mathbf{y}_i, \mathbf{X}_i, \mathbf{Z}_i)$ is 
$$
    \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma_0^2) = - \frac{n_i}{2} \log (2\pi) - \frac{1}{2} \log \det \boldsymbol{\Omega}_i - \frac{1}{2} (\mathbf{y} - \mathbf{X}_i \boldsymbol{\beta})^T \boldsymbol{\Omega}_i^{-1} (\mathbf{y} - \mathbf{X}_i \boldsymbol{\beta}),
$$
where
$$
    \boldsymbol{\Omega}_i = \sigma^2 \mathbf{I}_{n_i} + \mathbf{Z}_i \boldsymbol{\Sigma} \mathbf{Z}_i^T = \sigma^2 \mathbf{I}_{n_i} + \mathbf{Z}_i \mathbf{L} \mathbf{L}^T \mathbf{Z}_i^T.
$$
Because the variance component parameter $\boldsymbol{\Sigma}$ has to be positive semidefinite. We prefer to use its Cholesky factor $\mathbf{L}$ as optimization variable. 

Given $m$ independent data tuples $(\mathbf{y}_i, \mathbf{X}_i, \mathbf{Z}_i)$, $i=1,\ldots,m$, we seek the maximum likelihood estimate (MLE) by maximizing the log-likelihood
$$
\ell(\boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma^2) = \sum_{i=1}^m \ell_i(\boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma^2).
$$
In this assignment, we use the nonlinear programming (NLP) approach for optimization. In HW7, we will derive an EM (expectation-maximization) algorithm for the same problem. There is also an MM (minorization-maximization) algorithm for the same problem; see [this article](https://doi.org/10.1080/10618600.2018.1529601).

In [7]:
# load necessary packages; make sure install them first
using BenchmarkTools, CSV, DataFrames, DelimitedFiles, Distributions
using LinearAlgebra, MathOptInterface, NLopt
using PrettyTables, Random, RCall
using Ipopt, MixedModels

const MOI = MathOptInterface

MathOptInterface

## Q1. (Optional, 30 bonus pts) Derivatives

NLP optimization solvers expect users to provide at least a function for evaluating objective value. If users can provide further information such as gradient and Hessian, the NLP solvers will be more stable and converge faster. Automatic differentiation tools are becoming more powerful but cannot apply to all problems yet.

1. Show that the gradient of $\ell_i$ is
\begin{eqnarray*}
\nabla_{\boldsymbol{\beta}} \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2) &=& \mathbf{X}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i, \\
\nabla_{\sigma^2} \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2) &=& - \frac{1}{2} \operatorname{tr} (\boldsymbol{\Omega}_i^{-1}) + \frac{1}{2} \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-2} \mathbf{r}_i, \\
\frac{\partial}{\partial \mathbf{L}} \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2) &=& - \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L} + \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L},
\end{eqnarray*}
where $\mathbf{r}_i = \mathbf{y}_i - \mathbf{X}_i \boldsymbol{\beta}$. 

2. Derive the observed information matrix and the expected (Fisher) information matrix.

If you need a refresher on multivariate calculus, my [Biostat 216 lecture notes](https://ucla-biostat216-2019fall.github.io/slides/16-matrixcalc/16-matrixcalc.html) may be helpful.

**Sol.**

**Q1**
$$
\begin{aligned}
\nabla_\beta l_i\left(\beta, L, \sigma^2\right) & =\frac{d}{d \beta}\left(-\frac{1}{2}\left(y_i-X_i \beta\right)^{T} \Omega_i^{-1}\left(y_i-X_i \beta\right)\right) \\
& =\frac{d}{d \beta}\left(-\frac{1}{2}\left(y_i^{T} \Omega_i^{-1} y_i-2 \beta^{T} X_i^{T} \Omega_i^{-1} y_i+\beta^{T} X_i^{T} \Omega_i^{-1} X_i \beta\right)\right) \\
& =x_i^{T} \Omega_i^{-1} y_i-x_i^{T} \Omega_i^{-1} x_i \beta \\
& =x_i^{T} \Omega_i^{-1}\left(y_i-x_i \beta\right) \\
& =x_i^{T} \Omega_i^{-1} r_i \\
\nabla_{\sigma^2} l_i\left(\beta, L, \sigma^2\right) & =\frac{d}{d \sigma^2}\left(-\frac{1}{2} \log \operatorname{det} \Omega_i-\frac{1}{2}\left(y_i-x_i \beta\right)^{T} \Omega_i^{-1}\left(y_i-x_i \beta\right)\right) \\
& =\frac{d}{d \sigma^2}\left(-\frac{1}{2} \log \operatorname{det}\left(\sigma^2 I+z_i \Sigma z_i^{T}\right)-\frac{1}{2} r_i^{T} \Omega_i^{-1} r_i\right) \\
& =-\frac{1}{2} \frac{1}{\operatorname{det}\left(\Omega_i\right)} \cdot \operatorname{det}\left(\Omega_i\right) \cdot \operatorname{tr}\left(\Omega_i^{-1} \cdot I\right)-\frac{1}{2} \operatorname{tr}\left(\frac{d}{d \sigma^2} r_i^{T} \Omega_i^{-1} r_i\right) \\
& =-\frac{1}{2} \operatorname{tr}\left(\Omega_i^{-1}\right)-\frac{1}{2} \operatorname{tr}\left(\Omega_i^{-1} r_i r_i^{T} \Omega_i^{-1}\right) \\
& =-\frac{1}{2} \operatorname{tr}\left(\Omega_i^{-1}\right)-\frac{1}{2} \operatorname{tr}\left(r_i^{T} \Omega_i^{-2} r_i\right)
\end{aligned}
$$
$$
\begin{aligned}
\nabla_L l_i\left(\beta, L, \sigma^2\right) & =\frac{d}{d L}\left(-\frac{1}{2} \log \operatorname{det} \Omega_i-\frac{1}{2}\left(y_i-x_i \beta\right)^{T} \Omega_i^{-1}\left(y_i-x_i \beta\right)\right) \\
& =-\frac{1}{2}\left[\Omega_i^{-1} \frac{d}{d L}\left(\sigma^2 I+Z_i L L^{T} Z_i^{T}\right)+\frac{d}{d L} \operatorname{tr}\left(r_i^{T} \Omega_i^{-1} r_i\right)\right] \\
& =-\frac{1}{2}\left[\Omega_i^{-1}\left(Z_i L \otimes Z_i+\left(Z_i \otimes Z_i L\right) K_{q q}\right)-\Omega_i^{-1} r_i r_i^{T} \Omega_i^{-1}\left(Z_i L \otimes Z_i+\left(Z_i \otimes Z_i L\right) K_{q q}\right)\right] \\
& =-\frac{1}{2}\left[V e c\left(\Omega_i^{-1}\right)\left(Z_i L \otimes Z_i+\left(Z_i \otimes Z_i L\right) K_{q q}\right)-V_{e c}\left(\Omega_i^{-1} r_i r_i^{T} \Omega_i^{-1}\right)\left(Z_i L \otimes Z_i+\left(Z_i \otimes Z_i L\right) K_{q q}\right)\right] \\
& =-\frac{1}{2}\left[2\left(V_{e c} L^{T} Z_i^{T} \Omega_i^{-1} Z_i\right)^{T}-2\left(V_{e c} L^{T} Z_i^{T} \Omega_i^{-1} r_i r^{T} \Omega^{-1} Z_i\right)\right] \\
& =-Z_i^{T} \Omega_i^{-1} Z_i L+Z_i^{T} \Omega_i^{-1} r_i r_i^{T} \Omega_i^{-1} Z_i L
\end{aligned}
$$

**Q2**

We can calculate the Hessian of the log-likelihood,

$$
\begin{aligned}
& H\left(\beta, L, \sigma^2\right)=\sum_{i=1}^n\left[\begin{array}{lll}
\nabla_\beta^2 \ell_i\left(\beta, L, \sigma^2\right) & \nabla_\beta \nabla_L \ell_i\left(\beta, L, \sigma^2\right) & \nabla_\beta \nabla_{\sigma^2} \ell_i\left(\beta, L, \sigma^2\right) \\
\nabla_L \nabla_\beta \ell_i\left(\beta, L, \sigma^2\right) & \nabla_L^2 \ell_i\left(\beta, L, \sigma^2\right) & \nabla_L \nabla_{\sigma^2} \ell_i\left(\beta, L, \sigma^2\right) \\
\nabla_{\sigma^2} \nabla_\beta \ell_i\left(\beta, L, \sigma^2\right) & \nabla_{\sigma^2} \nabla_L \ell_i\left(\beta, L, \sigma^2\right) & \nabla_{\sigma^2}^2 \ell_i\left(\beta, L, \sigma^2\right)
\end{array}\right]
\end{aligned}
$$

Then we can obtain the expected (Fisher) information matrix.
$$
\begin{aligned}
I\left(\beta, L, \sigma^2\right)=-E\left(H\left(\beta, L, \sigma^2\right)\right)
\end{aligned}
$$

## Q2. (20 pts) Objective and gradient evaluator for a single datum

We expand the code from HW3 to evaluate both objective and gradient. I provide my code for HW3 below as a starting point. You do _not_ have to use this code. If your come up faster code, that's even better. 

In [8]:
# define a type that holds an LMM datum
struct LmmObs{T <: AbstractFloat}
    # data
    y          :: Vector{T}
    X          :: Matrix{T}
    Z          :: Matrix{T}
    # arrays for holding gradient
    ∇β         :: Vector{T}
    ∇σ²        :: Vector{T}
    ∇Σ         :: Matrix{T}    
    # working arrays
    # TODO: whatever intermediate arrays you may want to pre-allocate
    yty        :: T
    xty        :: Vector{T}
    zty        :: Vector{T}
    storage_p  :: Vector{T}
    storage_q  :: Vector{T}
    storage_q2  :: Vector{T}
    xtx        :: Matrix{T}
    ztx        :: Matrix{T}
    ztz        :: Matrix{T}
    storage_qq :: Matrix{T}
    storage_qq2 :: Matrix{T}
    storage_qq3 :: Matrix{T}
end

"""
    LmmObs(y::Vector, X::Matrix, Z::Matrix)

Create an LMM datum of type `LmmObs`.
"""
function LmmObs(
        y::Vector{T}, 
        X::Matrix{T}, 
        Z::Matrix{T}
    ) where T <: AbstractFloat
    n, p, q    = size(X, 1), size(X, 2), size(Z, 2)    
    ∇β         = Vector{T}(undef, p)
    ∇σ²        = Vector{T}(undef, 1)
    ∇Σ         = Matrix{T}(undef, q, q)    
    yty        = abs2(norm(y))
    xty        = transpose(X) * y
    zty        = transpose(Z) * y    
    storage_p  = Vector{T}(undef, p)
    storage_q  = Vector{T}(undef, q)
    storage_q2  = Vector{T}(undef, q)
    xtx        = transpose(X) * X
    ztx        = transpose(Z) * X
    ztz        = transpose(Z) * Z
    storage_qq = similar(ztz)
    storage_qq2 = similar(ztz)
    storage_qq3 = similar(ztz)
    LmmObs(y, X, Z, ∇β, ∇σ², ∇Σ, 
        yty, xty, zty, storage_p, storage_q, storage_q2,
        xtx, ztx, ztz, storage_qq, storage_qq2, storage_qq3)
end

"""
    logl!(obs::LmmObs, β, L, σ², needgrad=false)

Evaluate the log-likelihood of a single LMM datum at parameter values `β`, `L`, 
and `σ²`. If `needgrad==true`, then `obs.∇β`, `obs.∇Σ`, and `obs.σ² are filled 
with the corresponding gradient.
"""
function logl!(
        obs      :: LmmObs{T}, 
        β        :: Vector{T}, 
        L        :: Matrix{T}, 
        σ²       :: T,
        needgrad :: Bool = true
    ) where T <: AbstractFloat
    n, p, q = size(obs.X, 1), size(obs.X, 2), size(obs.Z, 2)
    ####################
    # Evaluate objective
    ####################    
    # form the q-by-q matrix: M = σ² * I + Lt Zt Z L
    copy!(obs.storage_qq, obs.ztz)
    BLAS.trmm!('L', 'L', 'T', 'N', T(1), L, obs.storage_qq) # O(q^3)
    BLAS.trmm!('R', 'L', 'N', 'N', T(1), L, obs.storage_qq) # O(q^3)
    copy!(obs.storage_qq2, obs.storage_qq)
    @inbounds for j in 1:q
        obs.storage_qq[j, j] += σ²
    end
    # cholesky on M = σ² * I + Lt Zt Z L
    LAPACK.potrf!('U', obs.storage_qq) # O(q^3)
    # storage_q = (Mchol.U') \ (Lt * (Zt * res))
    BLAS.gemv!('N', T(-1), obs.ztx, β, T(1), copy!(obs.storage_q, obs.zty)) # O(pq)
    BLAS.trmv!('L', 'T', 'N', L, obs.storage_q)    # O(q^2)
    BLAS.trsv!('U', 'T', 'N', obs.storage_qq, obs.storage_q) # O(q^3)
    # l2 norm of residual vector
    copy!(obs.storage_p, obs.xty)
    rtr  = obs.yty +
        dot(β, BLAS.gemv!('N', T(1), obs.xtx, β, T(-2), obs.storage_p))
    # assemble pieces
    logl::T = n * log(2π) + (n - q) * log(σ²) # constant term
    @inbounds for j in 1:q
        logl += 2log(obs.storage_qq[j, j])
    end
    qf    = abs2(norm(obs.storage_q)) # quadratic form term
    logl += (rtr - qf) / σ² 
    logl /= -2
    ###################
    # Evaluate gradient
    ###################    
    if needgrad
        #######  ∇β   ########
        # xtr
        BLAS.gemv!('N', T(-1), obs.xtx, β, T(1), copy!(obs.∇β, obs.xty))
        # storage_q = L M -1 Lt Zt r
        BLAS.trsv!('U', 'N', 'N', obs.storage_qq, obs.storage_q)
        BLAS.trmv!('L', 'N', 'N', L, obs.storage_q)
        # get ∇β
        BLAS.gemv!('T', T(- 1 / σ²), obs.ztx, obs.storage_q, T(1 / σ²), obs.∇β)
        
        #######  ∇σ²   ########
        # -1/2 tr (Ω-1)
        LAPACK.potrs!('U', obs.storage_qq, obs.storage_qq2)
        obs.∇σ²[1] = n
        obs.∇σ²[1] -= tr(obs.storage_qq2)
        # 1/2 rt Ω-2 r
        BLAS.gemm!('N', 'N', T(1), obs.ztz, obs.storage_q, T(0), obs.storage_q2)
        #get ∇σ² 
        obs.∇σ²[1] -= (rtr - 2qf + dot(obs.storage_q, obs.storage_q2)) / σ²
        obs.∇σ²[1] /= -2 * σ²
        
        #######  ∇L   ########
        # ztzL - ztz L M-1 Lt ztzL
        copy!(obs.storage_qq, obs.ztz)
        BLAS.trmm!('R', 'L', 'N', 'N', T(1), L, obs.storage_qq)
        #copy!(obs.storage_qq3, obs.storage_qq)
        #BLAS.gemm!('N', 'N', T(1 / σ²), obs.storage_qq, obs.storage_qq2, T(- 1 / σ²), obs.storage_qq)
        BLAS.gemm!('N', 'N', T(-1), obs.storage_qq, obs.storage_qq2, T(1), obs.storage_qq)
        obs.storage_qq ./= -σ²
        # Ztr - Ztz L M-1 Lt Ztr
        copy!(obs.storage_q2, obs.zty)
        BLAS.gemv!('N', T(-1), obs.ztx, β, T(1), obs.storage_q2)
        BLAS.gemm!('N', 'N', T(- 1 / σ²), obs.ztz, obs.storage_q, T(1 / σ²), obs.storage_q2)
        BLAS.gemm!('N', 'T', T(1), obs.storage_q2, obs.storage_q2, T(0), obs.storage_qq2)
        broadcast!(+, obs.∇Σ, obs.storage_qq, obs.storage_qq2)
    end    
    ###################
    # Return
    ###################        
    return logl    
end

logl!

In [9]:
?BLAS.gemm!

```
gemm!(tA, tB, alpha, A, B, beta, C)
```

Update `C` as `alpha*A*B + beta*C` or the other three variants according to [`tA`](@ref stdlib-blas-trans) and `tB`. Return the updated `C`.


It is a good idea to test correctness and efficiency of the single datum objective/gradient evaluator here. First generate the same data set as in [HW3](https://ucla-biostat-257.github.io/2023spring/hw/hw3/hw03.html).

In [10]:
Random.seed!(257)

# dimension
n, p, q = 2000, 5, 3
# predictors
X  = [ones(n) randn(n, p - 1)]
Z  = [ones(n) randn(n, q - 1)]
# parameter values
β  = [2.0; -1.0; rand(p - 2)]
σ² = 1.5
Σ  = fill(0.1, q, q) + 0.9I # compound symmetry 
L  = Matrix(cholesky(Symmetric(Σ)).L)
# generate y
y  = X * β + Z * rand(MvNormal(Σ)) + sqrt(σ²) * randn(n)

# form the LmmObs object
obs = LmmObs(y, X, Z);

### Correctness

In [11]:
@show logl = logl!(obs, β, L, σ², true)
@show obs.∇β
@show obs.∇σ²
@show obs.∇Σ;

logl = logl!(obs, β, L, σ², true) = -3256.1793358058253
obs.∇β = [0.26698108057144054, 41.6141833706737, -34.34664962312658, 36.10898510707527, 27.913948208793308]
obs.∇σ² = [1.6283715138404962]
obs.∇Σ = [-0.9279574741769041 0.06569178726963298 -0.3032856224663807; -0.034732237123220086 -0.9874050681701934 0.2836670688894426; -0.39496673140315974 0.1919905095430509 1.1793355043243674]


You will lose all 20 points if following statement throws `AssertionError`.

In [12]:
@assert abs(logl - (-3256.1793358058258)) < 1e-4
@assert norm(obs.∇β - [0.26698108057144054, 41.61418337067327, 
        -34.34664962312689, 36.10898510707527, 27.913948208793144]) < 1e-4
# @assert norm(obs.∇Σ - 
#     [-0.9464482950697888 0.057792444809492895 -0.30244127639188767; 
#         0.057792444809492895 -1.00087164917123 0.2845116557144694; 
#         -0.30244127639188767 0.2845116557144694 1.170040927259726]) < 1e-4
@assert abs(obs.∇σ²[1] - (1.6283715138412163)) < 1e-4

### Efficiency

Benchmark for evaluating objective only. This is what we did in HW3.

In [13]:
@benchmark logl!($obs, $β, $L, $σ², false)

BenchmarkTools.Trial: 10000 samples with 179 evaluations.
 Range (min … max):  601.117 ns …  1.261 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     607.821 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   614.601 ns ± 43.621 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▄█▅▂▂▁                                                       ▁
  ███████▇▇▇▅▆▅▅▅▃▄▆▅▅▅▅▄▄▅▅▅▄▅▅▄▄▅▄▄▅▅▆▅▁▅▅▅▄▄▅▃▃▅▃▃▁▁▄▁▃▄▃▄▅ █
  601 ns        Histogram: log(frequency) by time       856 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

Benchmark for objective + gradient evaluation.

In [14]:
bm_objgrad = @benchmark logl!($obs, $β, $L, $σ², true)

BenchmarkTools.Trial: 10000 samples with 10 evaluations.
 Range (min … max):  1.600 μs …   8.160 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.640 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.685 μs ± 256.736 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▅█▇  ▁            ▁                                         ▁
  ███▇▆█▅▅▅▆▇▇▆▆▅▅▃▇██▃▄▃▃▁▁▁▄▃▃▄▁▄▄▄▁▁▁▅▃▃▃▁▄▄▁▃▅▁▁▄▁▁▅▅▇█▇▆ █
  1.6 μs       Histogram: log(frequency) by time      3.01 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

My median runt time is 900ns. You will get full credit (10 pts) if the median run time is within 10μs.

In [15]:
#  The points you will get are
clamp(10 / (median(bm_objgrad).time / 1e3) * 10, 0, 10)

10.0

## Q3. LmmModel type

We create a `LmmModel` type to hold all data points and model parameters. Log-likelihood/gradient of a `LmmModel` object is simply the sum of log-likelihood/gradient of individual data points. 

In [16]:
# define a type that holds LMM model (data + parameters)
struct LmmModel{T <: AbstractFloat} <: MOI.AbstractNLPEvaluator
    # data
    data :: Vector{LmmObs{T}}
    # parameters
    β    :: Vector{T}
    L    :: Matrix{T}
    σ²   :: Vector{T}    
    # arrays for holding gradient
    ∇β   :: Vector{T}
    ∇σ²  :: Vector{T}
    ∇L   :: Matrix{T}
    # TODO: add whatever intermediate arrays you may want to pre-allocate
    xty  :: Vector{T}
    ztr2 :: Vector{T}
    xtx  :: Matrix{T}
    ztz2 :: Matrix{T}
end

"""
    LmmModel(data::Vector{LmmObs})

Create an LMM model that contains data and parameters.
"""
function LmmModel(obsvec::Vector{LmmObs{T}}) where T <: AbstractFloat
    # dims
    p    = size(obsvec[1].X, 2)
    q    = size(obsvec[1].Z, 2)
    # parameters
    β    = Vector{T}(undef, p)
    L    = Matrix{T}(undef, q, q)
    σ²   = Vector{T}(undef, 1)    
    # gradients
    ∇β   = similar(β)    
    ∇σ²  = similar(σ²)
    ∇L   = similar(L)
    # intermediate arrays
    xty  = Vector{T}(undef, p)
    ztr2 = Vector{T}(undef, abs2(q))
    xtx  = Matrix{T}(undef, p, p)
    ztz2 = Matrix{T}(undef, abs2(q), abs2(q))
    LmmModel(obsvec, β, L, σ², ∇β, ∇σ², ∇L, xty, ztr2, xtx, ztz2)
end

"""
    logl!(m::LmmModel, needgrad=false)

Evaluate the log-likelihood of an LMM model at parameter values `m.β`, `m.L`, 
and `m.σ²`. If `needgrad==true`, then `m.∇β`, `m.∇Σ`, and `m.σ² are filled 
with the corresponding gradient.
"""
function logl!(m::LmmModel{T}, needgrad::Bool = false) where T <: AbstractFloat
    logl = zero(T)
    if needgrad
        fill!(m.∇β , 0)
        fill!(m.∇L , 0)
        fill!(m.∇σ², 0)        
    end
    @inbounds for i in 1:length(m.data)
        obs = m.data[i]
        logl += logl!(obs, m.β, m.L, m.σ²[1], needgrad)
        if needgrad
            BLAS.axpy!(T(1), obs.∇β, m.∇β)
            BLAS.axpy!(T(1), obs.∇Σ, m.∇L)
            m.∇σ²[1] += obs.∇σ²[1]
        end
    end
    logl
end

logl!

## Q4. (20 pts) Test data

Let's generate a synthetic longitudinal data set to test our algorithm.

In [17]:
Random.seed!(257)

# dimension
m      = 1000 # number of individuals
ns     = rand(1500:2000, m) # numbers of observations per individual
p      = 5 # number of fixed effects, including intercept
q      = 3 # number of random effects, including intercept
obsvec = Vector{LmmObs{Float64}}(undef, m)
# true parameter values
βtrue  = [0.1; 6.5; -3.5; 1.0; 5; zeros(p - 5)]
σ²true = 1.5
σtrue  = sqrt(σ²true)
Σtrue  = Matrix(Diagonal([2.0; 1.2; 1.0; zeros(q - 3)]))
Ltrue  = Matrix(cholesky(Symmetric(Σtrue), Val(true), check=false).L)
# generate data
for i in 1:m
    # first column intercept, remaining entries iid std normal
    X = Matrix{Float64}(undef, ns[i], p)
    X[:, 1] .= 1
    @views Distributions.rand!(Normal(), X[:, 2:p])
    # first column intercept, remaining entries iid std normal
    Z = Matrix{Float64}(undef, ns[i], q)
    Z[:, 1] .= 1
    @views Distributions.rand!(Normal(), Z[:, 2:q])
    # generate y
    y = X * βtrue .+ Z * (Ltrue * randn(q)) .+ σtrue * randn(ns[i])
    # form a LmmObs instance
    obsvec[i] = LmmObs(y, X, Z)
end
# form a LmmModel instance
lmm = LmmModel(obsvec);

For later comparison with other software, we save the data into a text file `lmm_data.csv`. **Do not put this file in Git.** It takes 245.4MB storage.

In [18]:
(isfile("lmm_data.csv") && filesize("lmm_data.csv") == 245369685) || 
open("lmm_data.csv", "w") do io
    p = size(lmm.data[1].X, 2)
    q = size(lmm.data[1].Z, 2)
    # print header
    print(io, "ID,Y,")
    for j in 1:(p-1)
        print(io, "X" * string(j) * ",")
    end
    for j in 1:(q-1)
        print(io, "Z" * string(j) * (j < q-1 ? "," : "\n"))
    end
    # print data
    for i in eachindex(lmm.data)
        obs = lmm.data[i]
        for j in 1:length(obs.y)
            # id
            print(io, i, ",")
            # Y
            print(io, obs.y[j], ",")
            # X data
            for k in 2:p
                print(io, obs.X[j, k], ",")
            end
            # Z data
            for k in 2:q-1
                print(io, obs.Z[j, k], ",")
            end
            print(io, obs.Z[j, q], "\n")
        end
    end
end

### Correctness

Evaluate log-likelihood and gradient of whole data set at the true parameter values.

In [19]:
copy!(lmm.β, βtrue)
copy!(lmm.L, Ltrue)
lmm.σ²[1] = σ²true
@show obj = logl!(lmm, true)
@show lmm.∇β
@show lmm.∇σ²
@show lmm.∇L;

obj = logl!(lmm, true) = -2.8400684383699712e6
lmm.∇β = [41.065916707428414, 445.75120353964826, 157.01339922488782, -335.09977360732904, -895.6257448386676]
lmm.∇σ² = [-489.5361730380388]
lmm.∇L = [-209.4200329492506 28.592034724659797 26.73645089736996; 28.59193212127325 -23.006525933949508 -75.37427770753725; 26.736531182311897 -75.37422731528999 -56.459925427586846]


Test correctness. You will loss all 20 points if following code throws `AssertError`.

In [20]:
@assert abs(obj - (-2.840068438369969e6)) < 1e-4
@assert norm(lmm.∇β - [41.0659167074073, 445.75120353972426, 
        157.0133992249258, -335.09977360733626, -895.6257448385899]) < 1e-4
@assert norm(lmm.∇L - [-3.3982575935824837 31.32103842086001 26.73645089732865; 
        40.43528672997116 61.86377650461202 -75.37427770754684; 
        37.811051468724486 -82.56838431216435 -56.45992542754974]) < 1e-4
@assert abs(lmm.∇σ²[1] - (-489.5361730382465)) < 1e-4

LoadError: AssertionError: norm(lmm.∇L - [-3.3982575935824837 31.32103842086001 26.73645089732865; 40.43528672997116 61.86377650461202 -75.37427770754684; 37.811051468724486 -82.56838431216435 -56.45992542754974]) < 0.0001

### Efficiency

Test efficiency.

In [21]:
bm_model = @benchmark logl!($lmm, true)

BenchmarkTools.Trial: 2674 samples with 1 evaluation.
 Range (min … max):  1.696 ms …   5.294 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.740 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.865 ms ± 330.395 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▇▅▄▄▃▂▂▂▁ ▁▁                                                
  ███████████████▆▇▇▅▇▆▅▅▆▆▃▆▅▅▃▅▄▅▅▅▅▄▅▄▅▅▅▅▅▃▅▄▆▇▅▆▅▄▆▆▅▅▅▅ █
  1.7 ms       Histogram: log(frequency) by time      3.19 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

My median run time is 1.4ms. You will get full credit if your median run time is within 10ms. The points you will get are

In [22]:
clamp(10 / (median(bm_model).time / 1e6) * 10, 0, 10)

10.0

### Memory

You will lose 1 point for each 100 bytes memory allocation. So the points you will get is

In [23]:
clamp(10 - median(bm_model).memory / 100, 0, 10)

10.0

## Q5. (30 pts) Starting point

For numerical optimization, a good starting point is critical. Let's start $\boldsymbol{\beta}$ and $\sigma^2$ from the least sqaures solutions (ignoring intra-individual correlations)
\begin{eqnarray*}
\boldsymbol{\beta}^{(0)} &=& \left(\sum_i \mathbf{X}_i^T \mathbf{X}_i\right)^{-1} \left(\sum_i \mathbf{X}_i^T \mathbf{y}_i\right) \\
\sigma^{2(0)} &=& \frac{\sum_i \|\mathbf{r}_i^{(0)}\|_2^2}{\sum_i n_i} = \frac{\sum_i \|\mathbf{y}_i - \mathbf{X}_i \boldsymbol{\beta}^{(0)}\|_2^2}{\sum_i n_i}.
\end{eqnarray*}
To get a reasonable starting point for $\boldsymbol{\Sigma} = \mathbf{L} \mathbf{L}^T$, we can minimize the least squares criterion (ignoring the noise variance component)
$$
    \text{minimize} \sum_i \| \mathbf{r}_i^{(0)} \mathbf{r}_i^{(0)T} - \mathbf{Z}_i \boldsymbol{\Sigma} \mathbf{Z}_i^T \|_{\text{F}}^2.
$$
Derive the minimizer $\boldsymbol{\Sigma}^{(0)}$ (10 pts). 

We implement this start point strategy in the function `init_ls()`.

**Sol.**

To derive the minimizer $\boldsymbol{\Sigma}^{(0)}$, we can set $$0 = \frac{d}{d\Sigma} \sum_{i=1}^n\left\|{r}_i^{(0)} {r}_i^{(0) T}-{Z}_{{i}} {Z}_{{i}}^{{T}}\right\|_F^2$$ then we have,

$$
\begin{aligned}
0 &= \frac{d}{d\Sigma} \sum_{i=1}^n \operatorname{tr}\left[\left({r}_i^{(0)} {r}_i^{(0) T}-{Z}_{{i}} {\Sigma} {Z}_{{i}}^{{T}}\right)\left({r}_i^{(0)} {r}_i^{(0) T}-{Z}_{{i}} \Sigma {Z}_{{i}}^{{T}}\right)^T\right] \\
0 &= \frac{d}{d\Sigma} \sum_{i=1}^n \left[\operatorname { t r } \left(\left({r}_i^{(0)} {r}_i^{(0) T} {r}_i^{(0)} {r}_i^{(0) T}-2 \operatorname{tr}\left({Z}_{{i}} {\Sigma} {Z}_{{i}}^{{T}} {r}_i^{(0)} {r}_i^{(0) T}\right)+\operatorname{tr}\left({Z}_{{i}} {\Sigma} {Z}_{{i}}^{{T}} {Z}_{{i}} {\Sigma} {Z}_{{i}}^{{T}}\right)\right]\right.\right. \\
0 &= \sum_{i=1}^n\left[-2\left({Z}_{{i}}^T {r}_i^{(0)} {r}_i^{(0) T} {Z}_{{i}}\right)+2\left({Z}_{{i}}^T {Z}_{{i}} \Sigma {Z}_{{i}}^T {Z}_{{i}}\right)\right] \\
\sum_{i=1}^n {Z}_{{i}}^T {r}_i^{(0)} {r}_i^{(0) T} {Z}_{{i}}&=\sum_{i=1}^n {Z}_{{i}}^T {Z}_{{i}} \Sigma {Z}_{{i}}^T {Z}_{{i}} \\
\sum_{i=1}^n\left({Z}_{{i}}^T {r}_i^{(0)} \otimes {Z}_{{i}}^T {r}_i^{(0)}\right)&=\sum_{i=1}^n\left({Z}_{{i}}^T {Z}_i \otimes {Z}_{{i}}^T {Z}_i\right) \operatorname{vec} \Sigma^{(0)}\\
\operatorname{vec} {\Sigma}^{(0)}&=\left(\sum_{i=1}^n {Z}_i^T {Z}_i \otimes {Z}_i^T {Z}_i\right)^{-1}\left(\sum_{i=1}^n {Z}_i^T {r}_i \otimes {Z}_i^T {r}_i\right)
\end{aligned}
$$

In [42]:
"""
    init_ls!(m::LmmModel)

Initialize parameters of a `LmmModel` object from the least squares estimate. 
`m.β`, `m.L`, and `m.σ²` are overwritten with the least squares estimates.
"""
function init_ls!(m::LmmModel{T}) where T <: AbstractFloat
    p, q = size(m.data[1].X, 2), size(m.data[1].Z, 2)
    
    ##### m.β ######
    m.xtx .= T(0)
    m.xty .= T(0)
    @inbounds for i in 1:length(m.data)
        m.xtx .+= m.data[i].xtx
        m.xty .+= m.data[i].xty
    end
    LAPACK.potrf!('L', m.xtx)
    LAPACK.potrs!('L', m.xtx, m.xty)
    m.β .= m.xty
    
    ##### σ^2 and L ##### 
    rsum = T(0)
    nsum = T(0)
    m.ztr2 .= T(0)
    m.ztz2 .= T(0)

    @inbounds for i in 1:length(m.data)
        #σ2
        obs = m.data[i]
        BLAS.gemv!('N', T(1), obs.xtx, m.β, T(0), obs.storage_p)
        rsum += obs.yty - 2 * dot(m.β, obs.xty) + dot(m.β, obs.storage_p)
        nsum += size(obs.X, 1)
        
        #Σ
        BLAS.gemv!('N', T(1), obs.ztx, m.β, T(0), obs.storage_q)
        obs.storage_q .= obs.zty - obs.storage_q
        BLAS.gemm!('N', 'T', T(1), obs.storage_q, obs.storage_q, T(0), obs.storage_qq)
        m.ztr2 .+= vec(obs.storage_qq)
        m.ztz2 .+= kron(obs.ztz, obs.ztz)
    end
    
    m.σ² .= rsum / nsum
    LAPACK.potrf!('L', m.ztz2)
    LAPACK.potrs!('L', m.ztz2, m.ztr2)
    m.L .= reshape(m.ztr2, q, q)
    LAPACK.potrf!('L', m.L)
    
    m
end

init_ls!

In [43]:
init_ls!(lmm)
@show logl!(lmm)
@show lmm.β
@show lmm.σ²
@show lmm.L;

logl!(lmm) = -3.362604934104436e6
lmm.β = [0.18207934611476334, 6.500480700993719, -3.497910784209159, 1.001113296229795, 5.0002519857919285]
lmm.σ² = [5.709004733413663]
lmm.L = [1.4069222734993234 0.07258461003916693 0.05717147035274016; 0.051591059013255315 1.131979211870369 -0.07707942768978573; 0.04063584138912108 -0.06994463586493149 0.9718256360134828]


### Correctness

Your start points should have a log-likelihood larger than -3.3627e6 (10 pts). The points you get are

In [44]:
# this is the points you get
(logl!(lmm) >  -3.3627e6) * 10

10

### Efficiency

The start point should be computed quickly. Otherwise there is no point using it as a starting point. My median run time is 175μs. You get full credit (10 pts) if the median run time is within 1ms.

In [45]:
bm_init = @benchmark init_ls!($lmm)

BenchmarkTools.Trial: 7801 samples with 1 evaluation.
 Range (min … max):  441.500 μs …   8.718 ms  ┊ GC (min … max): 0.00% … 91.54%
 Time  (median):     558.200 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   638.482 μs ± 543.354 μs  ┊ GC (mean ± σ):  7.52% ±  8.25%

  ██▅▂▂▂▁▂▁▁       ▂▄▅▄▄▅▅▆▆▄▄▃▂▁▁▁▁                            ▂
  █████████████▇█▇▇████████████████████▇▇██▇▆▆▅▅▆▇▆▆▆▄▆▅▆▆▇▅▅▁▅ █
  442 μs        Histogram: log(frequency) by time       1.13 ms <

 Memory estimate: 875.09 KiB, allocs estimate: 4002.

In [46]:
# this is the points you get
clamp(1 / (median(bm_init).time / 1e6) * 10, 0, 10)

10.0

## Q6. NLP via MathOptInterface.jl

We define the NLP problem using the modelling tool [MathOptInterface.jl](https://github.com/jump-dev/MathOptInterface.jl). Start-up code is given below. Modify if necessary to accomodate your own code.

In [47]:
"""
    fit!(m::LmmModel, solver=Ipopt.Optimizer())

Fit an `LmmModel` object by MLE using a nonlinear programming solver. Start point 
should be provided in `m.β`, `m.σ²`, `m.L`.
"""
function fit!(
        m :: LmmModel{T},
        solver = Ipopt.Optimizer()
    ) where T <: AbstractFloat
    p    = size(m.data[1].X, 2)
    q    = size(m.data[1].Z, 2)
    npar = p + ((q * (q + 1)) >> 1) + 1
    # prep the MOI
    MOI.empty!(solver)
    # set lower bounds and upper bounds of parameters
    # q diagonal entries of Cholesky factor L should be >= 0
    # σ² should be >= 0
    lb   = fill(0.0, q + 1)
    ub   = fill(Inf, q + 1)
    NLPBlock = MOI.NLPBlockData(MOI.NLPBoundsPair.(lb, ub), m, true)
    MOI.set(solver, MOI.NLPBlock(), NLPBlock)
    # start point
    params = MOI.add_variables(solver, npar)    
    par0   = Vector{T}(undef, npar)
    modelpar_to_optimpar!(par0, m)    
    for i in 1:npar
        MOI.set(solver, MOI.VariablePrimalStart(), params[i], par0[i])
    end
    MOI.set(solver, MOI.ObjectiveSense(), MOI.MAX_SENSE)
    # optimize
    MOI.optimize!(solver)
    optstat = MOI.get(solver, MOI.TerminationStatus())
    optstat in (MOI.LOCALLY_SOLVED, MOI.ALMOST_LOCALLY_SOLVED) || 
        @warn("Optimization unsuccesful; got $optstat")
    # update parameters and refresh gradient
    xsol = [MOI.get(solver, MOI.VariablePrimal(), MOI.VariableIndex(i)) for i in 1:npar]
    optimpar_to_modelpar!(m, xsol)
    logl!(m, true)
    m
end

"""
    modelpar_to_optimpar!(par, m)

Translate model parameters in `m` to optimization variables in `par`.
"""
function modelpar_to_optimpar!(
        par :: Vector,
        m   :: LmmModel
    )
    p = size(m.data[1].X, 2)
    q = size(m.data[1].Z, 2)
    # β
    copyto!(par, m.β)
    # L
    offset = p + 1
    @inbounds for j in 1:q, i in j:q
        par[offset] = m.L[i, j]
        offset += 1
    end
    # σ²
    par[end] = m.σ²[1]
    par
end

"""
    optimpar_to_modelpar!(m, par)

Translate optimization variables in `par` to the model parameters in `m`.
"""
function optimpar_to_modelpar!(
        m   :: LmmModel, 
        par :: Vector
    )
    p = size(m.data[1].X, 2)
    q = size(m.data[1].Z, 2)
    # β
    copyto!(m.β, 1, par, 1, p)
    # L
    fill!(m.L, 0)
    offset = p + 1
    @inbounds for j in 1:q, i in j:q
        m.L[i, j] = par[offset]
        offset   += 1
    end
    # σ²
    m.σ²[1] = par[end]    
    m
end

function MOI.initialize(
        m                  :: LmmModel, 
        requested_features :: Vector{Symbol}
    )
    for feat in requested_features
        if !(feat in MOI.features_available(m))
            error("Unsupported feature $feat")
        end
    end
end

MOI.features_available(m::LmmModel) = [:Grad, :Hess, :Jac]

function MOI.eval_objective(
        m   :: LmmModel, 
        par :: Vector
    )
    optimpar_to_modelpar!(m, par)
    logl!(m, false) # don't need gradient here
end

function MOI.eval_objective_gradient(
        m    :: LmmModel, 
        grad :: Vector, 
        par  :: Vector
    )
    p = size(m.data[1].X, 2)
    q = size(m.data[1].Z, 2)
    optimpar_to_modelpar!(m, par) 
    obj = logl!(m, true)
    # gradient wrt β
    copyto!(grad, m.∇β)
    # gradient wrt L
    offset = p + 1
    @inbounds for j in 1:q, i in j:q
        grad[offset] = m.∇L[i, j]
        offset += 1
    end
    # gradient with respect to σ²
    grad[end] = m.∇σ²[1]
    # return objective
    obj
end

function MOI.eval_constraint(m::LmmModel, g, par)
    p = size(m.data[1].X, 2)
    q = size(m.data[1].Z, 2)
    gidx   = 1
    offset = p + 1
    @inbounds for j in 1:q, i in j:q
        if i == j
            g[gidx] = par[offset]
            gidx   += 1
        end
        offset += 1
    end
    g[end] = par[end]
    g
end

function MOI.jacobian_structure(m::LmmModel)
    p    = size(m.data[1].X, 2)
    q    = size(m.data[1].Z, 2)
    row  = collect(1:(q + 1))
    col  = Int[]
    offset = p + 1
    for j in 1:q, i in j:q
        (i == j) && push!(col, offset)
        offset += 1
    end
    push!(col, offset)
    [(row[i], col[i]) for i in 1:length(row)]
end

MOI.eval_constraint_jacobian(m::LmmModel, J, par) = fill!(J, 1)

function MOI.hessian_lagrangian_structure(m::LmmModel)
    p    = size(m.data[1].X, 2)
    q    = size(m.data[1].Z, 2)    
    q◺   = ◺(q)
    # we work on the upper triangular part of the Hessian
    arr1 = Vector{Int}(undef, ◺(p) + ◺(q◺) + q◺ + 1)
    arr2 = Vector{Int}(undef, ◺(p) + ◺(q◺) + q◺ + 1)
    # Hββ block
    idx  = 1    
    for j in 1:p, i in 1:j
        arr1[idx] = i
        arr2[idx] = j
        idx      += 1
    end
    # HLL block
    for j in 1:q◺, i in 1:j
        arr1[idx] = p + i
        arr2[idx] = p + j
        idx      += 1
    end
    # HLσ² block
    for i in (p + 1):(p + q◺)
        arr1[idx] = i
        arr2[idx] = p + q◺ + 1
        idx      += 1
    end
    # Hσ²σ² block
    arr1[idx] = p + q◺ + 1
    arr2[idx] = p + q◺ + 1
    [(arr1[i], arr2[i]) for i in 1:length(arr1)]
end

function MOI.eval_hessian_lagrangian(
        m   :: LmmModel, 
        H   :: AbstractVector{T},
        par :: AbstractVector{T}, 
        σ   :: T, 
        μ   :: AbstractVector{T}
    ) where {T}    
    p  = size(m.data[1].X, 2)
    q  = size(m.data[1].Z, 2)    
    q◺ = ◺(q)
    optimpar_to_modelpar!(m, par)
    logl!(m, true, true)
    # Hββ block
    idx = 1    
    @inbounds for j in 1:p, i in 1:j
        H[idx] = m.Hββ[i, j]
        idx   += 1
    end
    # HLL block
    @inbounds for j in 1:q◺, i in 1:j
        H[idx] = m.HLL[i, j]
        idx   += 1
    end
    # HLσ² block
    @inbounds for j in 1:q, i in j:q
        H[idx] = m.Hσ²L[i, j]
        idx   += 1
    end
    # Hσ²σ² block
    H[idx] = m.Hσ²σ²[1, 1]
    lmul!(σ, H)
end

## Q7. (20 pts) Test drive

Now we can run any NLP solver (supported by MathOptInterface.jl) to compute the MLE. For grading purpose, let's use the `:LD_MMA` ([Method of Moving Asymptotes](https://nlopt.readthedocs.io/en/latest/NLopt_Algorithms/#mma-method-of-moving-asymptotes-and-ccsa)) algorithm in NLopt.jl.

In [48]:
# initialize from least squares
init_ls!(lmm)
println("objective value at starting point: ", logl!(lmm)); println()

# NLopt (LD_MMA) obj. val = -2.8400587866501966e6
NLopt_solver = NLopt.Optimizer()
MOI.set(NLopt_solver, MOI.RawOptimizerAttribute("algorithm"), :LD_MMA)
@time fit!(lmm, NLopt_solver)

println("objective value at solution: $(logl!(lmm)))")
println("solution values:")
@show lmm.β
@show lmm.σ²
@show lmm.L * transpose(lmm.L)
println("gradient @ solution:")
@show lmm.∇β
@show lmm.∇σ²
@show lmm.∇L
@show norm([lmm.∇β; vec(LowerTriangular(lmm.∇L)); lmm.∇σ²])

objective value at starting point: -3.362604934104436e6

  0.956957 seconds (1.18 M allocations: 78.514 MiB, 2.32% gc time, 83.89% compilation time)
objective value at solution: -2.840075956762544e6)
solution values:
lmm.β = [0.18173470989466922, 6.500383304372868, -3.499864658327308, 0.9997120347011923, 4.9992296875077225]
lmm.σ² = [1.4987379557095732]
lmm.L * transpose(lmm.L) = [1.5776129180819056 0.028608962562930315 0.025242504643198493; 0.028608962562930315 1.179773518906685 -0.04424120703768006; 0.025242504643198493 -0.04424120703768006 0.962766739398985]
gradient @ solution:
lmm.∇β = [0.06795050012899395, 0.3270457416028929, 0.4101125694586685, 0.03336844664084282, 0.007242655691538147]
lmm.∇σ² = [-1.260039275325333]
lmm.∇L = [0.19405601975067965 16.649425311040467 17.03453861314172; -0.039002431596475415 -2.141502391046919 -38.57985589991842; 0.021886673557233727 0.06092317502676984 -4.89631676377129]
norm([lmm.∇β; vec(LowerTriangular(lmm.∇L)); lmm.∇σ²]) = 5.520140697672979


5.520140697672979

### Correctness

You get 10 points if the following code does not throw `AssertError`.

In [53]:
# objective at solution should be close enough to the optimal
@assert logl!(lmm) > -2.840059e6
# gradient at solution should be small enough
@assert norm([lmm.∇β; vec(LowerTriangular(lmm.∇L)); lmm.∇σ²]) < 0.1

LoadError: AssertionError: logl!(lmm) > -2.840059e6

### Efficiency

My median run time is 50ms. You get 10 points if your median time is within 1s(=1000ms).

In [54]:
NLopt_solver = NLopt.Optimizer()
MOI.set(NLopt_solver, MOI.RawOptimizerAttribute("algorithm"), :LD_MMA)
bm_mma = @benchmark fit!($lmm, $(NLopt_solver)) setup=(init_ls!(lmm))

BenchmarkTools.Trial: 33 samples with 1 evaluation.
 Range (min … max):  148.913 ms … 156.220 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     150.989 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   151.583 ms ±   1.963 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

          ▃▃ ▃  ▃  █           ▃                                 
  ▇▁▇▇▁▇▁▇██▁█▁▇█▇▁█▇▇▁▁▇▁▇▁▁▁▁█▇▇▁▁▁▁▁▁▇▇▁▇▁▁▁▁▁▁▇▁▇▁▁▁▁▁▁▁▁▇▇ ▁
  149 ms           Histogram: frequency by time          156 ms <

 Memory estimate: 26.41 KiB, allocs estimate: 799.

In [55]:
# this is the points you get
clamp(1 / (median(bm_mma).time / 1e9) * 10, 0, 10)

10.0

## Q8. (10 pts) Gradient free vs gradient-based methods

Advantage of using a modelling tool such as MathOptInterface.jl is that we can easily switch the backend solvers. For a research problem, we never know beforehand which solver works best. 

Try different solvers in the NLopt.jl and Ipopt.jl packages. Compare the results in terms of run times (the shorter the better), objective values at solution (the larger the better), and gradients at solution (closer to 0 the better). Summarize what you find.

See this [page](https://nlopt.readthedocs.io/en/latest/NLopt_Algorithms/) for the descriptions of algorithms in NLopt.

Documentation for the Ipopt can be found [here](https://coin-or.github.io/Ipopt/).  

In [56]:
# vector of solvers to compare
solvers = ["NLopt (LN_COBYLA, gradient free)", "NLopt (LD_MMA, gradient-based)", 
    "Ipopt (L-BFGS)"]

function setup_solver(s::String)
    if s == "NLopt (LN_COBYLA, gradient free)"
        solver = NLopt.Optimizer()
        MOI.set(solver, MOI.RawOptimizerAttribute("algorithm"), :LN_COBYLA)
    elseif s == "NLopt (LD_MMA, gradient-based)"
        solver = NLopt.Optimizer()
        MOI.set(solver, MOI.RawOptimizerAttribute("algorithm"), :LD_MMA)
    elseif s == "Ipopt (L-BFGS)"
        solver = Ipopt.Optimizer()
        MOI.set(solver, MOI.RawOptimizerAttribute("print_level"), 0)
        MOI.set(solver, MOI.RawOptimizerAttribute("hessian_approximation"), "limited-memory")
        MOI.set(solver, MOI.RawOptimizerAttribute("tol"), 1e-6)
    elseif s == "Ipopt (use FIM)"
        # Ipopt (use Hessian) obj val = -2.8400587866468e6
        solver = Ipopt.Optimizer()
        MOI.set(solver, MOI.RawOptimizerAttribute("print_level"), 0)        
    else
        error("unrecognized solver $s")
    end
    solver
end

# containers for results
runtime = zeros(length(solvers))
objvals = zeros(length(solvers))
gradnrm = zeros(length(solvers))

for i in 1:length(solvers)
    solver = setup_solver(solvers[i])
    bm = @benchmark fit!($lmm, $solver) setup = (init_ls!(lmm))
    runtime[i] = median(bm).time / 1e9
    objvals[i] = logl!(lmm, true)
    gradnrm[i] = norm([lmm.∇β; vec(LowerTriangular(lmm.∇L)); lmm.∇σ²])
end

LoadError: UndefVarError: `◺` not defined

In [57]:
# display results
pretty_table(
    hcat(solvers, runtime, objvals, gradnrm),
    header = ["Solver", "Runtime", "Log-Like", "Gradiant Norm"],
    formatters = (ft_printf("%5.2f", 2), ft_printf("%8.8f", 3:4))
    )

┌──────────────────────────────────┬─────────┬───────────────────┬───────────────┐
│                           Solver │ Runtime │          Log-Like │ Gradiant Norm │
├──────────────────────────────────┼─────────┼───────────────────┼───────────────┤
│ NLopt (LN_COBYLA, gradient free) │    0.60 │ -2840080.63551096 │  364.42460726 │
│   NLopt (LD_MMA, gradient-based) │    0.15 │ -2840075.95676254 │    5.52014070 │
│                   Ipopt (L-BFGS) │    0.00 │        0.00000000 │    0.00000000 │
└──────────────────────────────────┴─────────┴───────────────────┴───────────────┘


## Q9. (10 pts) Compare with existing art

Let's compare our method with lme4 package in R and MixedModels.jl package in Julia. Both lme4 and MixedModels.jl are developed mainly by Doug Bates. Summarize what you find.

In [58]:
method  = ["My method", "lme4", "MixedModels.jl"]
runtime = zeros(3)  # record the run times
loglike = zeros(3); # record the log-likelihood at MLE

### Your approach

In [59]:
solver = setup_solver("NLopt (LD_MMA, gradient-based)")
bm_257 = @benchmark fit!($lmm, $solver) setup=(init_ls!(lmm))
runtime[1] = (median(bm_257).time) / 1e9
loglike[1] = logl!(lmm)

-2.840075956762544e6

### lme4 

In [67]:
R"""
library(lme4)
library(readr)
library(magrittr)

testdata <- read_csv("lmm_data.csv")
"""

Rows: 1744977 Columns: 8
-- Column specification --------------------------------------------------------
Delimiter: ","
dbl (8): ID, Y, X1, X2, X3, X4, Z1, Z2

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.


                                                                                                                   
└ @ RCall C:\Users\larry\.julia\packages\RCall\LWzAQ\src\io.jl:172


RObject{VecSxp}
# A tibble: 1,744,977 x 8
      ID        Y     X1      X2     X3      X4      Z1      Z2
   <dbl>    <dbl>  <dbl>   <dbl>  <dbl>   <dbl>   <dbl>   <dbl>
 1     1   9.52    0.202 -0.463   0.798  0.734   0.685  -0.570 
 2     1  24.4     1.59  -1.95    1.20   1.43    1.64    0.369 
 3     1  -1.99    0.378 -0.0367  1.63  -1.15   -0.818   2.83  
 4     1 -17.4    -1.88   0.375  -0.498 -0.253   1.56    1.68  
 5     1  -0.0704  0.658 -0.165   0.780 -1.23   -0.0288 -1.09  
 6     1  -0.853   0.458 -0.313  -0.512 -0.800  -0.331   1.98  
 7     1  -1.80    0.220  0.328   1.32  -1.01   -0.363  -0.0703
 8     1   5.88    1.30   0.889  -0.854  0.0714 -0.658  -0.0339
 9     1  -9.21   -1.43  -0.522  -0.119 -0.580  -0.155  -1.89  
10     1 -11.3    -0.468 -0.700   0.872 -1.82    1.80    0.492 
# i 1,744,967 more rows
# i Use `print(n = ...)` to see more rows


In [ ]:
R"""
rtime <- system.time(mmod <- 
  lmer(Y ~ X1 + X2 + X3 + X4 + (1 + Z1 + Z2 | ID), testdata, REML = FALSE))
"""

RObject{RealSxp}
   user  system elapsed 
  43.90    8.46   82.81 


In [ ]:
R"""
rtime <- rtime["elapsed"]
summary(mmod)
rlogl <- logLik(mmod)
"""
runtime[2] = @rget rtime
loglike[2] = @rget rlogl;

### MixedModels.jl



In [82]:
testdata = CSV.File("lmm_data.csv", types = Dict(1=>String)) |> DataFrame

Row,ID,Y,X1,X2,X3,X4,Z1,Z2
,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,9.52102,0.201821,-0.463234,0.797731,0.73357,0.685476,-0.569622
2,1,24.4063,1.58557,-1.94608,1.19787,1.43149,1.63962,0.369053
3,1,-1.99215,0.378332,-0.0367002,1.63072,-1.15031,-0.817843,2.83422
4,1,-17.4233,-1.8826,0.374561,-0.49786,-0.253248,1.56433,1.67857
5,1,-0.0704245,0.658283,-0.165487,0.77951,-1.22763,-0.0287779,-1.09172
6,1,-0.853357,0.457784,-0.313387,-0.512299,-0.800278,-0.330632,1.97609
7,1,-1.80061,0.220461,0.327879,1.32209,-1.01336,-0.362947,-0.0703055
8,1,5.88119,1.30135,0.88884,-0.853941,0.0714372,-0.658202,-0.0338648
9,1,-9.20504,-1.43248,-0.521638,-0.119287,-0.579596,-0.154869,-1.88707


In [83]:
mj = fit(MixedModel, @formula(Y ~ X1 + X2 + X3 + X4 + (1 + Z1 + Z2 | ID)), testdata)
bm_mm = @benchmark fit(MixedModel, @formula(Y ~ X1 + X2 + X3 + X4 + (1 + Z1 + Z2 | ID)), $testdata)
loglike[3] = loglikelihood(mj)
runtime[3] = median(bm_mm).time / 1e9

┌ Warning: ProgressMeter by default refresh meters with additional information in IJulia via `IJulia.clear_output`, which clears all outputs in the cell. 
│  - To prevent this behaviour, do `ProgressMeter.ijulia_behavior(:append)`. 
│  - To disable this warning message, do `ProgressMeter.ijulia_behavior(:clear)`.
└ @ ProgressMeter C:\Users\larry\.julia\packages\ProgressMeter\sN2xr\src\ProgressMeter.jl:618
Minimizing 176 	 Time: 0:00:00 ( 1.32 ms/it)


0.4722747

In [84]:
display(bm_mm)
mj

BenchmarkTools.Trial: 11 samples with 1 evaluation.
 Range (min … max):  449.533 ms … 515.701 ms  ┊ GC (min … max): 1.88% … 9.66%
 Time  (median):     472.275 ms               ┊ GC (median):    5.05%
 Time  (mean ± σ):   473.591 ms ±  21.002 ms  ┊ GC (mean ± σ):  5.97% ± 2.38%

  ▁  ▁▁ ▁          ▁  ▁▁ ▁                  █                 ▁  
  █▁▁██▁█▁▁▁▁▁▁▁▁▁▁█▁▁██▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  450 ms           Histogram: frequency by time          516 ms <

 Memory estimate: 841.83 MiB, allocs estimate: 6392.

|             |    Est. |     SE |        z |      p |   σ_ID |
|:----------- | -------:| ------:| --------:| ------:| ------:|
| (Intercept) |  0.1815 | 0.0445 |     4.08 | <1e-04 | 1.4082 |
| X1          |  6.5004 | 0.0009 |  7000.80 | <1e-99 |        |
| X2          | -3.4999 | 0.0009 | -3768.92 | <1e-99 |        |
| X3          |  0.9997 | 0.0009 |  1078.16 | <1e-99 |        |
| X4          |  4.9992 | 0.0009 |  5391.08 | <1e-99 |        |
| Z2          |         |        |          |        | 0.9713 |
| Z1          |         |        |          |        | 1.1319 |
| Residual    |  1.2242 |        |          |        |        |


### Summary

In [85]:
pretty_table(
    hcat(method, runtime, loglike),
    header = ["Method", "Runtime", "Log-Like"],
    formatters = (ft_printf("%5.2f", 2), ft_printf("%8.6f", 3))
    )

┌────────────────┬─────────┬─────────────────┐
│         Method │ Runtime │        Log-Like │
├────────────────┼─────────┼─────────────────┤
│      My method │    0.15 │ -2840075.956763 │
│           lme4 │   82.81 │ -2840058.786648 │
│ MixedModels.jl │    0.47 │ -2840058.786716 │
└────────────────┴─────────┴─────────────────┘


## Q9. Be proud of yourself

Go to your resume/cv and claim you have experience performing analysis on complex longitudinal data sets with millions of records. And you beat current software by XXX fold.